In [0]:
# https://www.youtube.com/watch?v=NzmoPqte4V4
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.applications import MobileNet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam

Using TensorFlow backend.


## Define the model with Transfer learning of imagenet

In [0]:
base_model = MobileNet(weights='imagenet', include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)
preds = Dense(2, activation='softmax')(x)

Instructions for updating:
Colocations handled automatically by placer.


/usr/local/lib/python3.6/dist-packages/keras_applications/mobilenet.py:208: UserWarning: MobileNet shape is undefined. Weights for input shape (224, 224) will be loaded.
  warnings.warn('MobileNet shape is undefined.'


17227776/17225924 [==============================] - 2s 0us/step


In [0]:
model = Model(inputs=base_model.input, outputs=preds)

In [0]:
for layer in model.layers[:20]:
  layer.trainable = False
for layer in model.layers[20:]:
  layer.trainable = True # Only train in the last layer

## Upload dataset
* sign up for a Kaggle account at https://www.kaggle.com.
* go to the 'Account' tab of your user profile https://www.kaggle.com/
* select 'Create API Token'.
* This will trigger the download of kaggle.json, a file containing your API credentials

In [0]:
!pip install -U -q kaggle
!mkdir -p ~/.kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"phiztee","key":"71b8f9f82db0c8f264c1e3d3eed254b8"}'}

In [0]:
!cp kaggle.json ~/.kaggle/
!kaggle datasets download -d paultimothymooney/chest-xray-pneumonia

 98% 1.13G/1.15G [00:16<00:00, 72.4MB/s]
100% 1.15G/1.15G [00:16<00:00, 72.8MB/s]


In [0]:
!apt install pv
!unzip -o /content/chest-xray-pneumonia.zip  | pv -l >/dev/null
os.remove('chest-xray-pneumonia.zip')
!unzip -o /content/chest_xray.zip  | pv -l >/dev/null
os.remove('chest_xray.zip')

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'apt autoremove' to remove it.
Suggested packages:
  doc-base
The following NEW packages will be installed:
  pv
0 upgraded, 1 newly installed, 0 to remove and 11 not upgraded.
Need to get 48.3 kB of archives.
After this operation, 123 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 pv amd64 1.6.6-1 [48.3 kB]
Fetched 48.3 kB in 1s (44.2 kB/s)
Selecting previously unselected package pv.
(Reading database ... 130812 files and directories currently installed.)
Preparing to unpack .../archives/pv_1.6.6-1_amd64.deb ...
Unpacking pv (1.6.6-1) ...
Setting up pv (1.6.6-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
2.00  0:00:09 [ 218m/s] [  <=>                                                 ]
11.8k 0:00:14 [ 839 /s] [              

## Train Model

In [0]:
train_datagen=ImageDataGenerator(preprocessing_function=preprocess_input) #included in our dependencies

train_generator=train_datagen.flow_from_directory('/content/chest_xray/train/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=32,
                                                  class_mode='categorical', shuffle=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator=test_datagen.flow_from_directory('/content/chest_xray/test/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=16,
                                                  class_mode='categorical', shuffle=True)

valid_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

valid_generator=valid_datagen.flow_from_directory('/content/chest_xray/val/',
                                                  target_size=(224,224),
                                                  color_mode='rgb',
                                                  batch_size=16,
                                                  class_mode='categorical', shuffle=True)

model.compile(optimizer='Adam',loss='categorical_crossentropy',metrics=['accuracy'])
# Adam optimizer
# loss function will be categorical cross entropy
# evaluation metric will be accuracy

step_size_train=train_generator.n//train_generator.batch_size
model.fit_generator(generator=train_generator,
                   steps_per_epoch=step_size_train,
                   epochs=5,
                   validation_data=test_generator,
                   validation_steps=test_generator.samples // test_generator.batch_size)

Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
163/163 [==============================] - 68s 418ms/step - loss: 0.1176 - acc: 0.9624 - val_loss: 1.9246 - val_acc: 0.6779
Epoch 2/5
163/163 [==============================] - 61s 372ms/step - loss: 0.0521 - acc: 0.9818 - val_loss: 0.9930 - val_acc: 0.8237
Epoch 3/5
163/163 [==============================] - 61s 375ms/step - loss: 0.0440 - acc: 0.9831 - val_loss: 2.2555 - val_acc: 0.6987
Epoch 4/5
163/163 [==============================] - 60s 368ms/step - loss: 0.0213 - acc: 0.9914 - val_loss: 1.6278 - val_acc: 0.7163
Epoch 5/5
163/163 [==============================] - 59s 363ms/step - loss: 0.0258 - acc: 0.9912 - val_loss: 1.4204 - val_acc: 0.7468


## Save the weight

In [0]:
model.save('chest-xray-pneumonia.h5')    
!zip -r model.zip 'chest-xray-pneumonia.h5'

updating: chest-xray-pneumonia.h5 (deflated 9%)


## Test the model

In [0]:
from keras.models import load_model
new_model = load_model("/content/chest-xray-pneumonia.h5")

def get_rez(pic):
  img = image.load_img(pic, target_size=(224,224))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x = preprocess_input(x)
  p_good,p_ill = np.around(new_model.predict(x), decimals=2)[0]
  return{'p_good':p_good,'p_ill':p_ill}

In [0]:
ill_path = "/content/chest_xray/test/PNEUMONIA/" 
good_path = "/content/chest_xray/test/NORMAL/" 

ill_pic = ill_path + os.listdir(ill_path)[0]
good_pic = good_path + os.listdir(good_path)[1]

print(get_rez(ill_pic))
print(get_rez(good_pic))

{'p_good': 0.0, 'p_ill': 1.0}
{'p_good': 0.59, 'p_ill': 0.41}
